# Higuchi Fractal Dimension

<div style="font-size: 13px; font-family: 'Times New Roman', Times, serif; background-color: #181818; color: #D0D0D0; padding: 20px; border-radius: 8px; margin: 10px; display: flex; flex-wrap: nowrap; justify-content: space-between;">
    <!-- Column 1 -->
    <div style="flex: 1; margin-right: 10px;">
        <h2>Introduction</h2>
        <p>
            This code calculates the Higuchi Fractal Dimension (HFD) of EEG signals for multiple channels.
            Fractal dimensions are used to quantify the complexity of a data series, which is particularly useful in neuroscience for understanding the complexity of brain activity.
        </p>
    </div>
    <!-- Column 2 -->
    <div style="flex: 1; margin-left: 10px;">
        <p>
            The code uses the Higuchi Fractal Dimension algorithm, which is computationally efficient and particularly useful for analyzing time series data. It works by evaluating the length of the curve created by the data points at different scales.
        </p>
        \[
        L_k = \frac{N-1}{\left\lfloor \frac{N-m}{k} \right\rfloor k} \sum_{m=0}^{k-1} \left[ \sum_{i=1}^{\left\lfloor \frac{N-m}{k} \right\rfloor} \left| x[m+i \cdot k] - x[m+i \cdot k - k] \right| \right]
        \]
        \[
        HFD = \frac{\log(L_{k2}/L_{k1})}{\log(k2/k1)}
        \]
        <p>
            Here, \( L_k \) is the length of the curve for a given \( k \) (delay or time offset). \( N \) is the length of the data, \( x \) is the time series data, and \( m \) is the initial time point for each \( k \).
        </p>
        <p>
            Finally, the HFD values for each channel are stored in the list <code>hfd_values</code>.
        </p>
    </div>
</div>

In [8]:
import numpy as np

# Load EEG data
EEG_data = np.load('/home/vincent/AAA_projects/MVCS/Neuroscience/eeg_data_with_channels.npy', allow_pickle=True)

# Adjust the list to match your data's channels
eeg_channels = ['Fp1', 'Fpz', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6',
                'M1', 'T7', 'C3', 'Cz', 'C4', 'T8', 'M2', 'CP5', 'CP1', 'CP2', 'CP6',
                'P7', 'P3', 'Pz', 'P4', 'P8', 'POz', 'O1', 'Oz', 'O2']

def higuchi_fd(data, k_max):
    """Compute Higuchi Fractal Dimension of a time series.
    
    Parameters:
    data : list or np.array
        One-dimensional time series
    k_max : int
        Maximum delay (time offset)
        
    Returns:
    hfd : float
        Higuchi Fractal Dimension
    """
    N = len(data)
    L = []
    
    x = np.asarray(data)
    
    for k in range(1, k_max):
        Lk = []
        
        for m in range(0, k):
            Lkm = 0
            for i in range(1, int((N-m)/k)):
                Lkm += abs(x[m+i*k] - x[m+i*k-k])
            Lkm = Lkm*(N - 1)/(((N - m)/k)*k)
            Lk.append(Lkm)
            
        L.append(np.log(np.mean(Lk)))
    
    hfd = np.polyfit(np.log(range(1, k_max)), L, 1)[0]
    
    return hfd

hfd_values = []

for channel_data in EEG_data.T:  # .T because we need to iterate over channels
    hfd_channel = higuchi_fd(channel_data, k_max=10)  # Adjust k_max as needed
    hfd_values.append(hfd_channel)

print("Higuchi Fractal Dimensions for each channel:", hfd_values)

Higuchi Fractal Dimensions for each channel: [-0.36005497045124624, -0.23633437232263893, -0.37112149677594536, -0.08754714948970373, -0.11035468235736252, -0.13707288326129607, -0.3934048831939664, -0.24673709351805825, -0.0710953944331469, -0.10275985039837075, -0.3467667431744175, -0.24990709690042587, -0.21426902542688608, -0.09127876391896302, -0.08507749371463456, -0.25722852525504636, -0.2805514117387538, -0.2224151766390623, -0.2099877581545126, -0.0799982792792008, -0.20791211613648958, -0.33103053273893573, -0.2269624863539871, -0.2363840209397604, -0.2704812265220445, -0.31618623070687657, -0.2329294151814016, -0.20944694956243734, -0.2115303234095175, -0.19603356897036747, -0.20279111854623105, -0.2125008482366303]


# Save the results

In [9]:
# Save the results to a text file
results_file_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Fractal Analysis/Higuchi_Fractal_Dimensions.txt'
with open(results_file_path, 'w') as f:
    for channel, hfd in zip(eeg_channels, hfd_values):
        f.write(f"Channel {channel}: {hfd}\n")

print("Results saved to:", results_file_path)

Results saved to: /home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Fractal Analysis/Higuchi_Fractal_Dimensions.txt


In [4]:
import numpy as np
import pandas as pd

# Load the results from the given file
results_file_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Fractal Analysis/Higuchi_Fractal_Dimensions.txt'
with open(results_file_path, 'r') as f:
    # Each line is of the form 'Channel <channel_name>: <HFD value>'
    hfd_values = [float(line.split(':')[-1].strip()) for line in f]

# Generate CNN Feature
# Reshape HFD values into a 4x8 matrix
cnn_features = np.array(hfd_values).reshape(4, 8)

# Generate RNN Feature
# Treat the HFD values as a sequence
rnn_features = np.array(hfd_values).reshape(1, -1)  # 1 sequence of HFD values for each channel

# Save the features
save_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Features'
np.save(f"{save_path}/cnn_fractal.npy", cnn_features)
np.save(f"{save_path}/rnn_fractal.npy", rnn_features)

# Print shape and head using pandas
# For CNN feature
cnn_df = pd.DataFrame(cnn_features)
print("\nCNN Feature Shape:")
print(cnn_df.shape)
print("\nCNN Feature Head:")
print(cnn_df.head())

# For RNN feature
rnn_df = pd.DataFrame(rnn_features)
print("\nRNN Feature Shape:")
print(rnn_df.shape)
print("\nRNN Feature Head:")
print(rnn_df.head())



CNN Feature Shape:
(4, 8)

CNN Feature Head:
          0         1         2         3         4         5         6  \
0 -0.360055 -0.236334 -0.371121 -0.087547 -0.110355 -0.137073 -0.393405   
1 -0.071095 -0.102760 -0.346767 -0.249907 -0.214269 -0.091279 -0.085077   
2 -0.280551 -0.222415 -0.209988 -0.079998 -0.207912 -0.331031 -0.226962   
3 -0.270481 -0.316186 -0.232929 -0.209447 -0.211530 -0.196034 -0.202791   

          7  
0 -0.246737  
1 -0.257229  
2 -0.236384  
3 -0.212501  

RNN Feature Shape:
(1, 32)

RNN Feature Head:
         0         1         2         3         4         5         6   \
0 -0.360055 -0.236334 -0.371121 -0.087547 -0.110355 -0.137073 -0.393405   

         7         8        9   ...        22        23        24        25  \
0 -0.246737 -0.071095 -0.10276  ... -0.226962 -0.236384 -0.270481 -0.316186   

         26        27       28        29        30        31  
0 -0.232929 -0.209447 -0.21153 -0.196034 -0.202791 -0.212501  

[1 rows x 32 columns]
